<a href="https://colab.research.google.com/github/hadil-kh/alx-pre_course/blob/master/voice_health_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/mydrive")

Mounted at /content/mydrive


In [2]:
%cd /content/mydrive/MyDrive/Projet e-health

/content/mydrive/MyDrive/Projet e-health


In [3]:
!nvidia-smi

Thu Aug 14 11:14:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
!pip install gTTS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [5]:
!pip install SpeechRecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 30.2 MB/s eta 0:00:00


In [6]:
!pip install pydub ffmpeg-python

In [7]:
global tokenizer, pipeline_model

In [8]:
import gradio as gr
import os
import tempfile
from transformers import AutoTokenizer, pipeline
import torch
import transformers
from gtts import gTTS
import speech_recognition as sr
from pydub import AudioSegment

In [9]:
# Configuration for Colab
AUTO_PLAY_AUDIO = True  # Set to False if audio autoplay causes issues

In [10]:
global tokenizer, pipeline

model = "llama-2-7b-custom100-FineTuned"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [11]:
# Welcome message
WELCOME_MESSAGE = """🩺 Hello! I'm your virtual medical assistant.

I can help you with:
• 💊 Medication information
• 🔍 Symptom analysis
• 📋 General health advice
• 🚨 Emergency guidance

How can I assist you today?"""

In [12]:
def chat_response(message):
    """Generate appropriate medical response"""
    system_message = """You are a professional and compassionate medical assistant.
    Provide clear, helpful responses while advising to consult a doctor when necessary.
    Be empathetic and precise in your medical responses."""

    prompt = f"[INST] <<SYS>>\n{system_message}\n<<SYS>>\n\n{message}[/INST]"

    try:
        sequences = pipeline(  # Changé de pipeline_model à pipeline
            prompt,
            do_sample=True,
            top_k=10,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
            max_length=1000,
            temperature=0.7,
        )

        for seq in sequences:
            bot_message = seq['generated_text'].replace(prompt, '').split('[/INST]')[-1]

        return bot_message.strip()
    except Exception as e:
        return f"Sorry, an error occurred. Please try again. Error: {str(e)}"

In [13]:
def speech_to_text(audio_path):
    """Convert audio to text with improved error handling"""
    if audio_path is None:
        return "No audio detected"

    r = sr.Recognizer()
    r.energy_threshold = 300
    r.pause_threshold = 0.8

    try:
        # Load and process audio
        audio = AudioSegment.from_file(audio_path)

        # Normalize audio for better recognition
        audio = audio.normalize()

        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp_file:
            audio.export(tmp_file.name, format="wav")

            with sr.AudioFile(tmp_file.name) as source:
                # Adjust for ambient noise
                r.adjust_for_ambient_noise(source, duration=0.5)
                audio_data = r.record(source)

                try:
                    # Try English first, then other languages
                    try:
                        text = r.recognize_google(audio_data, language="en-US")
                    except sr.UnknownValueError:
                        text = r.recognize_google(audio_data, language="fr-FR")

                    os.unlink(tmp_file.name)
                    return text

                except sr.UnknownValueError:
                    return "I couldn't understand your audio. Please speak more clearly."
                except sr.RequestError as e:
                    return f"Speech recognition service error: {str(e)}"

    except Exception as e:
        return f"Audio processing error: {str(e)}"
    finally:
        # Clean up temporary files
        if 'tmp_file' in locals() and os.path.exists(tmp_file.name):
            try:
                os.unlink(tmp_file.name)
            except:
                pass

In [14]:
def text_to_speech(text, lang='en'):
    """Convert text to audio with multilingual support"""
    try:
        if not text or len(text.strip()) == 0:
            return None

        # Clean text for TTS
        clean_text = text.replace('*', '').replace('#', '').strip()

        if len(clean_text) > 500:
            clean_text = clean_text[:500] + "..."

        with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as tmp_file:
            tts = gTTS(text=clean_text, lang=lang, slow=False)
            tts.save(tmp_file.name)
            return tmp_file.name

    except Exception as e:
        print(f"Text-to-speech error: {e}")
        return None

In [15]:
def respond_text(message, chat_history):
    """Process text messages"""
    if not message or message.strip() == "":
        return "", chat_history

    bot_message = chat_response(message)
    chat_history.append((message, bot_message))
    return "", chat_history

def respond_audio(audio, chat_history):
    """Process audio messages"""
    if audio is None:
        chat_history.append((None, "⚠️ Please record your voice message first"))
        return "", chat_history, None

    # Convert audio to text
    text = speech_to_text(audio)

    if any(error in text for error in ["couldn't understand", "Error", "No audio"]):
        chat_history.append((None, f"🔊 {text}"))
        return "", chat_history, None

    # Generate response
    bot_message = chat_response(text)
    chat_history.append((f"🎤 You said: {text}", bot_message))

    # Generate audio response
    audio_response = text_to_speech(bot_message)

    return "", chat_history, audio_response

In [16]:
def generate_voice_response(chat_history):
    """Generate voice response for the last message"""
    if chat_history and len(chat_history) > 0:
        last_message = chat_history[-1][1]
        audio_file = text_to_speech(last_message)
        return audio_file, gr.Audio(visible=True)
    return None, gr.Audio(visible=False)

In [17]:
# Enhanced CSS for medical mobile interface
medical_mobile_css = """
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700&display=swap');

:root {
    --medical-primary: #2563eb;
    --medical-secondary: #1e40af;
    --medical-accent: #3b82f6;
    --medical-success: #059669;
    --medical-warning: #d97706;
    --medical-danger: #dc2626;
    --medical-light: #f8fafc;
    --medical-white: #ffffff;
    --medical-gray-50: #f9fafb;
    --medical-gray-100: #f3f4f6;
    --medical-gray-200: #e5e7eb;
    --medical-gray-400: #9ca3af;
    --medical-gray-500: #6b7280;
    --medical-gray-600: #4b5563;
    --medical-gray-700: #374151;
    --medical-gray-900: #111827;
    --shadow-sm: 0 1px 2px 0 rgba(0, 0, 0, 0.05);
    --shadow-md: 0 4px 6px -1px rgba(0, 0, 0, 0.1);
    --shadow-lg: 0 10px 15px -3px rgba(0, 0, 0, 0.1);
}

* {
    box-sizing: border-box;
}

.gradio-container {
    font-family: 'Inter', -apple-system, BlinkMacSystemFont, sans-serif !important;
    max-width: 100% !important;
    padding: 0 !important;
    background: var(--medical-light);
    min-height: 100vh;
}

/* Modern medical header */
.medical-header {
    background: linear-gradient(135deg, var(--medical-primary) 0%, var(--medical-secondary) 100%);
    color: white;
    padding: 24px 20px;
    text-align: center;
    position: sticky;
    top: 0;
    z-index: 100;
    box-shadow: var(--shadow-lg);
}

.medical-header h1 {
    margin: 0;
    font-size: 24px;
    font-weight: 700;
    letter-spacing: -0.025em;
}

.medical-header .subtitle {
    margin: 8px 0 0;
    font-size: 14px;
    color: rgba(255,255,255,0.9);
    font-weight: 400;
}

.medical-header .status-indicator {
    display: inline-block;
    width: 8px;
    height: 8px;
    background: var(--medical-success);
    border-radius: 50%;
    margin-right: 8px;
    animation: pulse 2s infinite;
}

@keyframes pulse {
    0%, 100% { opacity: 1; }
    50% { opacity: 0.5; }
}

/* Enhanced chat area */
.medical-chatbot {
    min-height: 50vh !important;
    max-height: 50vh !important;
    overflow-y: auto !important;
    background: var(--medical-white);
    border: none !important;
    border-radius: 16px;
    margin: 16px;
    box-shadow: var(--shadow-md);
    padding: 20px !important;
}

/* Avatar styles */
.avatar img {
    width: 32px !important;
    height: 32px !important;
    border-radius: 50% !important;
    object-fit: contain !important;
    background: var(--medical-primary);
    padding: 4px;
}

.user-avatar {
    background: var(--medical-accent) !important;
}

.bot-avatar {
    background: var(--medical-success) !important;
}

/* User and bot messages */
.user-message {
    background: linear-gradient(135deg, var(--medical-primary), var(--medical-accent)) !important;
    color: white !important;
    padding: 16px 20px !important;
    border-radius: 20px 20px 8px 20px !important;
    margin: 12px 0 !important;
    max-width: 85% !important;
    margin-left: auto !important;
    box-shadow: var(--shadow-sm);
    font-weight: 500;
}

.bot-message {
    background: var(--medical-gray-50) !important;
    color: var(--medical-gray-900) !important;
    padding: 16px 20px !important;
    border-radius: 20px 20px 20px 8px !important;
    margin: 12px 0 !important;
    max-width: 85% !important;
    border-left: 4px solid var(--medical-accent);
    box-shadow: var(--shadow-sm);
    line-height: 1.6;
}

/* Emoji styling in messages */
.bot-message emoji, .user-message emoji {
    font-size: 1.2em !important;
    vertical-align: middle !important;
    margin: 0 2px !important;
}

/* Medical buttons */
.medical-btn {
    background: var(--medical-primary) !important;
    color: white !important;
    border: none !important;
    padding: 14px 24px !important;
    border-radius: 12px !important;
    font-weight: 600 !important;
    font-size: 14px !important;
    transition: all 0.2s ease !important;
    box-shadow: var(--shadow-sm);
    width: 100%;
    margin: 8px 0 !important;
    display: flex !important;
    align-items: center !important;
    justify-content: center !important;
    gap: 8px !important;
}

.medical-btn .icon {
    font-size: 1.2em !important;
}

.medical-btn:hover {
    background: var(--medical-secondary) !important;
    transform: translateY(-1px);
    box-shadow: var(--shadow-md);
}

.medical-btn:active {
    transform: translateY(0);
}

/* Emergency button */
.emergency-btn {
    background: var(--medical-danger) !important;
    animation: emergency-pulse 2s infinite;
}

@keyframes emergency-pulse {
    0%, 100% { box-shadow: 0 0 0 0 rgba(220, 38, 38, 0.4); }
    50% { box-shadow: 0 0 0 10px rgba(220, 38, 38, 0); }
}

/* Input area */
.medical-input {
    border: 2px solid var(--medical-gray-200) !important;
    border-radius: 12px !important;
    padding: 16px 20px !important;
    font-size: 16px !important;
    transition: all 0.2s ease !important;
    background: var(--medical-white);
    margin: 12px 16px !important;
}

.medical-input:focus {
    border-color: var(--medical-primary) !important;
    box-shadow: 0 0 0 3px rgba(37, 99, 235, 0.1) !important;
    outline: none !important;
}

/* Tabs */
.medical-tabs {
    background: var(--medical-white);
    border-radius: 12px;
    margin: 16px;
    box-shadow: var(--shadow-sm);
    overflow: hidden;
}

.tab-nav {
    display: flex !important;
}

.tab-nav button {
    background: var(--medical-gray-50) !important;
    color: var(--medical-gray-600) !important;
    border: none !important;
    padding: 16px 24px !important;
    font-weight: 500 !important;
    transition: all 0.2s ease !important;
    flex: 1 !important;
    display: flex !important;
    align-items: center !important;
    justify-content: center !important;
    gap: 8px !important;
}

.tab-nav button .icon {
    font-size: 1.2em !important;
}

.tab-nav button.selected {
    background: var(--medical-primary) !important;
    color: white !important;
}

/* Audio area */
.audio-container {
    background: var(--medical-gray-50);
    border: 2px dashed var(--medical-gray-200);
    border-radius: 12px;
    padding: 24px;
    text-align: center;
    margin: 16px;
    transition: all 0.2s ease;
}

.audio-container:hover {
    border-color: var(--medical-primary);
    background: rgba(37, 99, 235, 0.05);
}

.audio-record-btn {
    background: var(--medical-success) !important;
    width: 80px !important;
    height: 80px !important;
    border-radius: 50% !important;
    display: flex !important;
    align-items: center !important;
    justify-content: center !important;
    margin: 0 auto 16px !important;
    font-size: 24px !important;
}

/* Status indicators */
.status-card {
    background: var(--medical-white);
    border-radius: 12px;
    padding: 16px;
    margin: 16px;
    box-shadow: var(--shadow-sm);
    border-left: 4px solid var(--medical-success);
}

.status-online {
    color: var(--medical-success);
    font-weight: 600;
    display: flex;
    align-items: center;
    gap: 8px;
}

/* Medical disclaimer */
.medical-disclaimer {
    background: var(--medical-warning);
    color: white;
    padding: 16px;
    margin: 16px;
    border-radius: 12px;
    font-size: 12px;
    text-align: center;
    line-height: 1.5;
}

/* Responsive design */
@media (max-width: 768px) {
    .gradio-container {
        padding: 0 !important;
    }

    .medical-chatbot {
        margin: 8px;
        min-height: 45vh !important;
        max-height: 45vh !important;
    }

    .medical-header {
        padding: 16px 12px;
    }

    .medical-header h1 {
        font-size: 20px;
    }

    .medical-input {
        margin: 8px !important;
        font-size: 16px !important;
    }

    .tab-nav button {
        padding: 12px 16px !important;
        font-size: 14px !important;
    }
}

/* Smooth animations */
.fade-in {
    animation: fadeIn 0.3s ease-in;
}

@keyframes fadeIn {
    from { opacity: 0; transform: translateY(10px); }
    to { opacity: 1; transform: translateY(0); }
}

/* Custom scrollbar */
.medical-chatbot::-webkit-scrollbar {
    width: 6px;
}

.medical-chatbot::-webkit-scrollbar-track {
    background: var(--medical-gray-100);
    border-radius: 3px;
}

.medical-chatbot::-webkit-scrollbar-thumb {
    background: var(--medical-gray-400);
    border-radius: 3px;
}

.medical-chatbot::-webkit-scrollbar-thumb:hover {
    background: var(--medical-gray-500);
}
"""

In [ ]:
with gr.Blocks(css=medical_mobile_css) as demo:
    gr.Markdown("<div class='medical-header'><h1>🩺 Virtual Medical Assistant</h1><p class='subtitle'><span class='status-indicator'></span>Online and ready to assist</p></div>")

    with gr.Tab("Text Chat"):
        chatbot = gr.Chatbot(label="Medical Chat", elem_classes="medical-chatbot")
        msg = gr.Textbox(label="Type your message", placeholder="Ask about symptoms, medications, etc...", elem_classes="medical-input")
        submit_btn = gr.Button("Send", elem_classes="medical-btn")
        submit_btn.click(respond_text, [msg, chatbot], [msg, chatbot])

    with gr.Tab("Voice Chat"):
        audio_input = gr.Audio(sources="microphone", type="filepath", label="Record your voice message")
        audio_output = gr.Audio(visible=False, label="Assistant's response")
        voice_chatbot = gr.Chatbot(label="Voice Chat", elem_classes="medical-chatbot")
        submit_audio_btn = gr.Button("Send Voice Message", elem_classes="medical-btn")
        submit_audio_btn.click(respond_audio, [audio_input, voice_chatbot], [msg, voice_chatbot, audio_output])

In [20]:
demo.launch()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d5a89700c22b860b2f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
